New chip seq notebook: 
- All of the datasets downloaded in previous notebook lacked information
- I think that the excel files have information I need, which means I need to redo all of the work from the other notebook with downloaded excel files as pandas dataframes
- 166 chipseq d. mel datasets from modENCODE
- Only downloading 159 of those because modMINE lacks information for the rest (see no_data list)

In [1]:
import pandas as pd
import urllib
import os
from urllib.request import urlopen

In [2]:
url = 'http://data.modencode.org/cgi-bin/cloud_list.pl?accessions=3393,3806,3825,3231,2625,2626,2637,3403,4078,3240,4080,5068,4082,4081,3959,5069,2638,2639,3395,3235,4974,5008,5070,5071,5072,5577,3229,3230,3402,3401,2640,2641,2642,3234,3236,3239,3241,3400,3398,4976,3824,3826,4089,3809,3238,3397,5028,3814,3245,3830,4119,4981,5257,3815,4982,3827,2629,2630,2635,2636,2633,2634,2627,2628,2631,2632,3391,3392,4091,3956,4936,5592,5111,5110,5590,5587,5112,5591,5264,5113,5593,4944,3390,5023,4998,4070,3242,5114,5575,5115,5594,3243,3394,3812,5004,5005,5116,5597,5574,5117,4094,5118,854,858,3233,862,863,856,849,857,855,985,859,860,861,5119,4074,4107,3957,3237,3813,3816,5120,5121,5598,4095,4096,3960,3808,4098,5576,5570,5579,5571,984,850,848,4352,5580,5569,5122,5123,5124,846,847,851,852,853,5125,5126,5599,4099,4071,3232,5568,5014,4103,5017,4069,4104,3958,4716,4105,4943,5127,5606,3396,5128,5024,5025,3399,4114,4113,3823,5029,5129&urls=1'
page = urlopen(url).read().decode('utf-8')
page = page.split('\n')

modid_list = []
for line in page:
    if not line.startswith('#'):
        modid = str(line.split()[0]) #gives an error but still works? 
        modid_list.append(modid)

IndexError: list index out of range

In [3]:
len(set(modid_list))

176

In [5]:
#define 18? ids that modmine has no data for (unknown reasons...)
no_data = ['2635', '2625', '2637', '2631', '3806', '2632', '3827', '3826', '3235', '3236', '3823', '3824', '3825', '2640', '3231', '3397', '3233', '2628'] 

In [11]:
PATH = '../../data/modENCODE_downloads_2ndtry/'
DWNLD1 = 'http://intermine.modencode.org/release-33/features.do?type=submission&action=export&format=csv&submission=modENCODE_'
DWNLD2 = '&feature=BindingSite'
os.makedirs(PATH, exist_ok=True)

done = []
for name in modid_list: 
    if name not in no_data: 
        if name not in done: 
            #try:
            urllib.request.urlretrieve(DWNLD1+name+DWNLD2, PATH+name) 
            done.append(name)
            #except
            
            

After the download:
- combine files to make one big file
- get as bed file
- do liftover
- intersect w gene bed file (one idea: if it's too hard to get these as beds, could use data from before and merge on location) 
- clean up table
- phantom peak overlap? 

In [78]:
#combine files into one: 
concat=[]
import glob
for fname in glob.glob('../../data/modENCODE_downloads_2ndtry/[0-9]*'):
    name = os.path.splitext(os.path.basename(fname))[0]
    if name not in no_data:
        df = pd.read_csv(fname)
        #df.astype(int)
        #df[2]
        concat.append(df)
        

In [84]:
big = pd.concat(concat)
big.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_big', sep='\t', na_rep=':.', header=None, index=False)

In [32]:
#iterate over liftovers to only take ones not in no data  
liftovers = []
for fname in glob.glob('../../data/modENCODE_downloads/modENCODE_*.liftover'):
    name = os.path.splitext(os.path.basename(fname))[0].split('_')[1]
    if name not in no_data: 
        if os.path.getsize(fname) > 0 :
            df = pd.read_table(fname, header=None)
            liftovers.append(df)
        else:
            print('Blank file:\t'+fname)
#create big liftover and save to file
liftframe = pd.concat(liftovers)
liftframe.to_csv('../../data/modENCODE_downloads_2ndtry/all_liftovers', sep='\t', header=None, index=False)

Blank file:	../../data/modENCODE_downloads/modENCODE_3807.liftover
Blank file:	../../data/modENCODE_downloads/modENCODE_3809.liftover
Blank file:	../../data/modENCODE_downloads/modENCODE_3810.liftover


In [36]:
#get gene file as a bed:
from pybedtools.featurefuncs import gff2bed
genes = BedTool('../../data/dmel-all-r6.12.gene_only.chr.gff')
genes_bed = genes.each(gff2bed, name_field='ID').saveas()

In [96]:
#make new id column (set to name/4th column)
#make this a bed and add chr to chromosome 
big.columns = ['DB_id','score','chrom','start','end','strand','modENCODE_id','type','name']

In [115]:
big['id'] = ['id='+str(x) for x in range(0, len(big.score))]

In [120]:
big['chrom'] = ['chr'+x for x in big['chrom']]

In [122]:
bigsub = big[['chrom','start','end','id']]
bigsub.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_big.bed', sep='\t', header=None, index=False)

In [123]:
#do liftover

In [132]:
#import liftover file
#do intersect
import pybedtools
from pybedtools import BedTool
#perform intersect on that
#bed intersect using big liftover file
with open('../../data/modENCODE_downloads_2ndtry/modENCODE_big.liftover') as f: 
    peaks_bed = BedTool(f)
    intersect = genes_bed.intersect(peaks_bed, wb=True).saveas()
    intdf = intersect.to_dataframe()
#then merge back on location to get all the info

In [141]:
trim = intdf[['chrom','start','end','name','score','strand','blockCount']]

In [142]:
merge = trim.merge(big, left_on='blockCount', right_on='id', how='left')
merge.head()

,chrom_x,start_x,end_x,name_x,score_x,strand_x,blockCount,DB_id,score_y,chrom_y,start_y,end_y,strand_y,modENCODE_id,type,name_y,id
0,chr2L,7528,8128,FBgn0031208,.,+,id=280047,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,strain,yellow cinnabar brown speck,id=280047
1,chr2L,7528,8128,FBgn0031208,.,+,id=280048,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,developmental stage,L3 stage larvae,id=280048
2,chr2L,7528,8128,FBgn0031208,.,+,id=280049,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,antibody,KW3-Trl-D2,id=280049
3,chr2L,7528,9484,FBgn0031208,.,+,id=383688,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,chr2L,5038,13342,0,modENCODE_3957,strain,Oregon-R-modENCODE,id=383688
4,chr2L,7528,9484,FBgn0031208,.,+,id=383689,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,chr2L,5038,13342,0,modENCODE_3957,developmental stage,Embryo 14-16hr OR,id=383689


In [143]:
merge.columns

Index(['chrom_x', 'start_x', 'end_x', 'name_x', 'score_x', 'strand_x',
       'blockCount', 'DB_id', 'score_y', 'chrom_y', 'start_y', 'end_y',
       'strand_y', 'modENCODE_id', 'type', 'name_y', 'id'],
      dtype='object')

In [147]:
another = merge[['chrom_x', 'start_x', 'end_x', 'name_x', 'strand_x', 'blockCount', 'DB_id', 'score_y','modENCODE_id', 'type', 'name_y']]
another.columns=['chrom','start','end','gene','strand','id','DB_id','score','modENCODE_id','type','name']      

In [149]:
len(another.modENCODE_id.unique())

150

In [150]:
another.head()

,chrom,start,end,gene,strand,id,DB_id,score,modENCODE_id,type,name
0,chr2L,7528,8128,FBgn0031208,+,id=280047,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,modENCODE_3830,strain,yellow cinnabar brown speck
1,chr2L,7528,8128,FBgn0031208,+,id=280048,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,modENCODE_3830,developmental stage,L3 stage larvae
2,chr2L,7528,8128,FBgn0031208,+,id=280049,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,modENCODE_3830,antibody,KW3-Trl-D2
3,chr2L,7528,9484,FBgn0031208,+,id=383688,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,modENCODE_3957,strain,Oregon-R-modENCODE
4,chr2L,7528,9484,FBgn0031208,+,id=383689,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,modENCODE_3957,developmental stage,Embryo 14-16hr OR


In [152]:
happy = another.set_index(
    ['modENCODE_id','DB_id','score','chrom','start','end','strand','gene','type']).pivot_table(
        columns='type', index=['DB_id','score','chrom','start','end','strand','modENCODE_id','gene'], 
        values='name',aggfunc='first')

In [154]:
happy = happy.reset_index()

In [157]:
happy.head()

type,DB_id,score,chrom,start,end,strand,modENCODE_id,gene,antibody,cell line,developmental stage,strain,target gene,tissue
0,15T_8_16_NW_GFP.1,50.62,chr2L,470602,471025,-,modENCODE_3402,FBgn0031258,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None
1,15T_8_16_NW_GFP.10,51.05,chr2L,3153151,3153697,-,modENCODE_3402,FBgn0011648,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None
2,15T_8_16_NW_GFP.101,106.48,chr2R,340719,341071,+,modENCODE_3402,FBgn0267428,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None
3,15T_8_16_NW_GFP.106,254.37,chr2R,1734635,1735241,+,modENCODE_3402,FBgn0087011,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None
4,15T_8_16_NW_GFP.107,55.08,chr2R,1878921,1879127,-,modENCODE_3402,FBgn0263780,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None


In [158]:
happy['target gene'].unique()

array([None, 'Eip74EF', 'Hr78', 'Stat92E', 'usp', 'lola'], dtype=object)

In [159]:
happy[happy['target gene'] == 'lola']

type,DB_id,score,chrom,start,end,strand,modENCODE_id,gene,antibody,cell line,developmental stage,strain,target gene,tissue
1273422,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000002,10.23,chr2L,550277,550523,-,modENCODE_5004,FBgn0005660,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273423,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000004,11.05,chr2L,594208,594448,-,modENCODE_5004,FBgn0010323,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273424,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000005,11.17,chr2L,688394,688716,-,modENCODE_5004,FBgn0000497,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273425,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000006,50.06,chr2L,779172,779655,+,modENCODE_5004,FBgn0031277,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273426,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000007,15.80,chr2L,949846,950848,+,modENCODE_5004,FBgn0028481,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273427,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000008,9.26,chr2L,950918,951458,+,modENCODE_5004,FBgn0028481,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273428,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000012,29.18,chr2L,1152443,1153227,-,modENCODE_5004,FBgn0261458,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273429,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000013,12.43,chr2L,1200238,1200524,+,modENCODE_5004,FBgn0263081,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273430,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000014,7.22,chr2L,1200746,1201060,+,modENCODE_5004,FBgn0040723,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None
1273431,lola_PJ_GFP_0_12h_embryo_ChIP_seq.gff_ID000016,7.78,chr2L,1618963,1619282,-,modENCODE_5004,FBgn0265814,GFP_ab290,None,Embryo 0-12 h,lola-RJ-GFP,lola,None


In [160]:
happy['strain'].unique()

array(['hairy-GFP-P[acman]-15T', 'eve-GFP-P[acman]-5T', '7T-CAD',
       'yellow cinnabar brown speck', 'Oregon-R-modENCODE', None,
       'Eip74EF-GFP', 'Hr78-GFP', 'R13-eve-YFP', 'Stat92E-GFP', 'Hr46-GFP',
       'usp-GFP', 'lola-RJ-GFP'], dtype=object)

In [181]:
#parse before GFP from DB_id: 

hopefully_genes = []
for item in happy.DB_id:
    new = new.split('.gff')[0]
    new = item.split('_GFP')[0]
    new = new.split('.')[0]
    hopefully_genes.append(new)

In [182]:
len(set(hopefully_genes))

105

In [183]:
set(hopefully_genes)

{'15T_8_16_NW',
 '5T_8_16_NW',
 '7T_W3L',
 'A_Female_Trem',
 'CG16778_0_8h_embryo_ChIP_seq',
 'CG8478_WPP_ChIP_seq',
 'CNC_16_24_seq',
 'CNC_AdultFemale_seq',
 'CTCF_14_16_hr_OR_embryo_Solexa',
 'Chro_Chriz_WR_14_16_hr_OR_embryo_Solexa',
 'Cl8_PolII_ChipSeq',
 'Dll_WPP_ChIP_seq',
 'E0_12h_SuVar3_9',
 'E0_8h_D',
 'E16_24h_HP1_Abcam',
 'E16_24h_HP1_Covance',
 'E16_24h_Hr39',
 'E16_24h_Kr_D2',
 'E16_24h_bon',
 'EcR_WPP_4_5h',
 'Eip74EF',
 'Fru_0_8_seq',
 'GAF_14_16_hr_OR_embryo_Solexa',
 'H1_14_16_hr_OR_embryo_Solexa',
 'HP1_8_16_seq',
 'HP1_wa191_D_mel_3rd_Instar_Larvae_Nuclei_Solexa',
 'HP1_wa191_OR_Head_Nuclei_Solexa',
 'HP1b_Q4114_14_16_hr_OR_embryo_Solexa',
 'HP1b_Q4114_OR_Head_Nuclei_Solexa',
 'HP1c__MO_462__OR_Head_Nuclei_Solexa_',
 'HP2__Ab2_90__D_mel_3rd_Instar_Larvae_Nuclei_Solexa',
 'HP2__Ab2_90__OR_Head_Nuclei_Solexa',
 'HP4_Q4072_14_16_hr_OR_embryo_Solexa',
 'H_0_8h_embryonic_ChIP_seq',
 'Hr78',
 'JHDM1_Q2634_14_16_hr_OR_embryo_Solexa',
 'JHDM1_Q2634_OR_Head_Nuclei_Solexa',
 

In [185]:
# The following two lines will be needed in every python script:
from intermine.webservice import Service
service = Service("http://intermine.modencode.org/release-33/service")

# Get a new query on the class (table) you will be querying:
query = service.new_query("Submission")

# The view specifies the output columns
query.add_view("DCCid", "assayFactor")

# Uncomment and edit the line below (the default) to select a custom sort order:
# query.add_sort_order("Submission.DCCid", "ASC")

# You can edit the constraint values below
query.add_constraint("experimentType", "=", "ChIP-seq", code = "A")
query.add_constraint("organism.commonName", "=", "fruit fly", code = "B")
#query.add_constraint("lab.name", "=", "Kevin White", code = "C")

# Uncomment and edit the code below to specify your own custom logic:
# query.set_logic("A and B and C")
table = []
for row in query.rows():
    newline = [row["DCCid"], row["assayFactor"]]
    table.append(newline)

In [191]:
gene_map = pd.DataFrame(table)
gene_map.columns = ['modENCODE_id','gene']
gene_map.head()

,modENCODE_id,gene
0,modENCODE_2626,Caudal
1,modENCODE_2627,HDAC4a
2,modENCODE_2629,HDAC11
3,modENCODE_2630,HDAC11
4,modENCODE_2633,HDAC3


In [194]:
withmap = happy.merge(gene_map, on='modENCODE_id', how='left')

In [196]:
withmap.columns = ['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'gene', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene', 'tissue', 'TF']

In [197]:
withmap.head()

,DB_id,score,chrom,start,end,strand,modENCODE_id,gene,antibody,cell line,developmental stage,strain,target gene,tissue,TF
0,15T_8_16_NW_GFP.1,50.62,chr2L,470602,471025,-,modENCODE_3402,FBgn0031258,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy
1,15T_8_16_NW_GFP.10,51.05,chr2L,3153151,3153697,-,modENCODE_3402,FBgn0011648,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy
2,15T_8_16_NW_GFP.101,106.48,chr2R,340719,341071,+,modENCODE_3402,FBgn0267428,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy
3,15T_8_16_NW_GFP.106,254.37,chr2R,1734635,1735241,+,modENCODE_3402,FBgn0087011,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy
4,15T_8_16_NW_GFP.107,55.08,chr2R,1878921,1879127,-,modENCODE_3402,FBgn0263780,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy


In [199]:
#add column for motif_FBgn
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',') 
addfbgn = withmap.merge(symbolmap, left_on='TF', right_on='SYMBOL', how='left')
addfbgn = addfbgn.rename(columns={'ENSEMBL': 'motif_FBgn'})

In [200]:
addfbgn

,DB_id,score,chrom,start,end,strand,modENCODE_id,gene,antibody,cell line,developmental stage,strain,target gene,tissue,TF,motif_FBgn,SYMBOL
0,15T_8_16_NW_GFP.1,50.62,chr2L,470602,471025,-,modENCODE_3402,FBgn0031258,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
1,15T_8_16_NW_GFP.10,51.05,chr2L,3153151,3153697,-,modENCODE_3402,FBgn0011648,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
2,15T_8_16_NW_GFP.101,106.48,chr2R,340719,341071,+,modENCODE_3402,FBgn0267428,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
3,15T_8_16_NW_GFP.106,254.37,chr2R,1734635,1735241,+,modENCODE_3402,FBgn0087011,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
4,15T_8_16_NW_GFP.107,55.08,chr2R,1878921,1879127,-,modENCODE_3402,FBgn0263780,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
5,15T_8_16_NW_GFP.108,53.52,chr2R,1994847,1995324,-,modENCODE_3402,FBgn0263780,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
6,15T_8_16_NW_GFP.11,295.70,chr2L,3301932,3302696,-,modENCODE_3402,FBgn0031516,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
7,15T_8_16_NW_GFP.122,99.40,chr2R,5247361,5247591,-,modENCODE_3402,FBgn0053554,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
8,15T_8_16_NW_GFP.126,54.64,chr2R,6692683,6693101,+,modENCODE_3402,FBgn0033092,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN
9,15T_8_16_NW_GFP.127,372.35,chr2R,6903226,6903955,+,modENCODE_3402,FBgn0050158,No Antibody Control,None,Embryo 8-16 h,hairy-GFP-P[acman]-15T,None,None,hairy,NaN,NaN


In [203]:
symbolmap[symbolmap.SYMBOL == 'hairy']

,ENSEMBL,SYMBOL


In [212]:
set(addfbgn.TF)

{'CG16778',
 'CG8478',
 'CP190',
 'CTCF',
 'Caudal',
 'Chriz',
 'DLL',
 'Dichaete',
 'EcR',
 'Eip74EF',
 'GAF',
 'GROUCHO',
 'H1',
 'HP1',
 'HP1a',
 'HP1b',
 'HP1c',
 'HP2',
 'HP4',
 'Hr39',
 'Hr78',
 'JHDM1',
 'JMJD2A/KDM4A',
 'Kruppel',
 'LSD1',
 'MBD-R2',
 'MOF',
 'NURF301',
 'POF',
 'Pc',
 'Psc',
 'RNA polymerase II',
 'RNA polymerase II CTD repeat YSPTSPS (phospho S2)',
 'RNA polymerase II CTD repeat YSPTSPS (phospho S5)',
 'RPD3',
 'Set2',
 'Smc3',
 'Stat92E',
 'Su(H)',
 'Su(Hw)',
 'Su(var)3-7',
 'Su(var)3-9',
 'Yki',
 'ZW5',
 'bonus',
 'cnc',
 'dMi-2',
 'dRING',
 'dSFMBT',
 'dm',
 'eve',
 'fru',
 'hairy',
 'hth',
 'lola',
 'mod(mdg4)',
 'pangolin',
 'prd',
 'psq',
 'scute',
 'tango',
 'trem',
 'usp'}

In [223]:
symbolmap2 = pd.read_table('../../data/fb_synonym.tsv', sep=' ', header=None)
symbolmap2.columns = ['gene','a','b']

In [227]:
for val in set(addfbgn.TF):
    if val in list(symbolmap2.a):
        print(val)

CG8478
Psc
Eip74EF
H1
Set2
Su(var)3-9
trem
Pc
psq
MBD-R2
eve
fru
HP1c
HP4
Hr39
CTCF
Stat92E
mod(mdg4)
Su(var)3-7
usp
hth
lola
EcR
prd
HP1b
Hr78
Su(H)
cnc


In [ ]:
#need to figure out which mapped and which didn't and then remerge...